In [1]:
import cv2
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML

import calibrate_and_undistort as cam_calibration
import color_and_grad_combined as gc_combined
import topview
import sliding_window

%matplotlib inline

In [2]:
#1. calibrate camera
cam_calibration.calibrate_camera()
M,Minv=topview.get_perspective_matrix()

camera_mtx,camera_dst=cam_calibration.get_cammtx_dstcoeff()
#print(M,Minv)
#print(camera_dst,camera_mtx)


(720, 1280, 3)


In [3]:
def region_of_interest(img, vertices):

    """
    Applies an image mask.
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    # defining a blank mask to start with
    mask = np.zeros_like(img)

    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    # filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [4]:
def process_image_test(img):

    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
    f.tight_layout()

    #2.undistort image
    undistorted_img=cam_calibration.undistort(img)
    ax1.imshow(undistorted_img)
    ax1.set_title("undistored image")

    #. apply perspective transform
    thresholded_warped = topview.get_top_view(undistorted_img)
    ax2.imshow(thresholded_warped)
    ax2.set_title("thresholded warped")

    #3.apply gradient and color thresolding
    thresholded_image=gc_combined.get_gc_combined(thresholded_warped)
    ax3.imshow(thresholded_image,cmap='gray')
    ax3.set_title("thresholded image")

    #grayed_thresholded=cv2.cvtColor(cv2.COLOR_HLS2GRAY)

    plt.show()

    return thresholded_warped

def process_image(img):
    
    # 2.undistort image  
    undistorted_img = cam_calibration.undistort(img)
    
    vertices = np.array([[(180,720), (1150,720),(720,430),(577,430)]], dtype = np.int32)
    #vertices = np.array([[ (150, 720), (1020, 720),(1020, 0), (0, 0)]], dtype=np.int32)
    roi=region_of_interest(undistorted_img,vertices)

    warped = topview.get_top_view(roi)
    
    thresholded_warped_image = gc_combined.get_gc_combined(warped)
    #sliding_window.plot_histogram(thresholded_warped_image)
    #unwraped_weighted_img,left_curvature,right_curvature=sliding_window.plot_histogram(np.uint8(thresholded_warped_image * 255),undistorted_img,Minv)
    unwraped_weighted_img,left_curvature,right_curvature=sliding_window.plot_histogram(thresholded_warped_image,undistorted_img,Minv)
    return unwraped_weighted_img
    #cl=np.dstack((thresholded_warped_image, thresholded_warped_image, thresholded_warped_image))*255
    #return cl
    
def process_video():
    print("video processing started..")
    
    final_output = 'test_output_video/output_roi.mp4'
    #final_output = 'test_output_video/final_debug-26-33_2.mp4'
    #final_output = 'test_output_video/debug-39-43_2.mp4'
    #final_output = 'test_output_video/submission_video.mp4'
    
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 5 seconds
    ##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)

    #clip1 = VideoFileClip("project_video.mp4")
    #clip1 = VideoFileClip("26-33.mp4")
    #clip1 = VideoFileClip("39-43.mp4")
    clip1 = VideoFileClip("project_video.mp4")#.subclip(23,25)
    undist_clip = clip1.fl_image(process_image)  # NOTE: this function expects color images!!
    
    %time undist_clip.write_videofile(final_output, audio=False)
    print("anupam")
    #undist_clip.write_videofile(final_output, audio=False)
    print("anupam")
    print("anupam")
    HTML("""
    <video width="960" height="540" controls>
      <source src="{0}">
    </video>
    """.format(final_output))

def pipeline_test_images():

    images = glob.glob('test_images/*.jpg')
    for fname in images:
        img = cv2.imread(fname)
        t=process_image(img)
        plt.imshow(t,cmap='gray')
        #histogram = np.sum(t[t.shape[0] // 2:, :], axis=0)
        #plt.plot(histogram)http://localhost:8888/notebooks/final_output.ipynb#
        print(fname,t.shape)
        plt.show()
        #sliding_window.plot_histogram(np.uint8(t*255),)

In [5]:
#pipeline_test_images()
process_video()

video processing started..
[MoviePy] >>>> Building video test_output_video/output_roi.mp4
[MoviePy] Writing video test_output_video/output_roi.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [07:49<00:00,  2.79it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_output_video/output_roi.mp4 

Wall time: 7min 51s
anupam
anupam
anupam
